In [2]:
import pandas as pd
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

MASK_CSV_PATH = '/inputdata/train_ship_segmentations_v2.csv'

In [3]:
print(os.getcwd()+MASK_CSV_PATH)


/Users/kate/Documents/KAGGLE_SHIPDETECTION/inputdata/train_ship_segmentations_v2.csv


In [4]:

masks_df = pd.read_csv(os.getcwd()+MASK_CSV_PATH)     
masks_df.head()

,ImageId,EncodedPixels
0,00003e153.jpg,NaN
1,0001124c7.jpg,NaN
2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...


In [10]:
masks_df.shape

(231723, 2)

In [19]:
masks_df_notna=masks_df[masks_df['EncodedPixels'].notna()]['EncodedPixels'].count()
masks_df_notna/masks_df['ImageId'].count()

0.3526753926023744

In [14]:
masks_df_na=masks_df[masks_df['EncodedPixels'].isna()].count()
masks_df_na

ImageId          150000
EncodedPixels         0
dtype: int64

In [ ]:
# Split the data
X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=VALIDATION_SPLIT, random_state=42)

X_train = tf.data.Dataset.from_tensor_slices(X_train)
y_train = tf.data.Dataset.from_tensor_slices(y_train)

X_val = tf.data.Dataset.from_tensor_slices(X_val)
y_val = tf.data.Dataset.from_tensor_slices(y_val)

#X_test = tf.data.Dataset.from_tensor_slices(X_test)

# Add labels to dataframe objects (one-hot-encoded)
train_dataset = tf.data.Dataset.zip((X_train, y_train))
val_dataset = tf.data.Dataset.zip((X_val, y_val))

# Apply the batch size to the dataset
batched_train_dataset = train_dataset.batch(BATCH_SIZE)
batched_val_dataset = val_dataset.batch(BATCH_SIZE)
#batched_test_dataset = X_test.batch(BATCH_SIZE)

# Adding autotune for pre-fetching
AUTOTUNE = tf.data.experimental.AUTOTUNE
batched_train_dataset = batched_train_dataset.prefetch(buffer_size=AUTOTUNE)
batched_val_dataset = batched_val_dataset.prefetch(buffer_size=AUTOTUNE)
#batched_test_dataset = batched_test_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
#check data
# View images and associated labels
for images, masks in batched_train_dataset.take(2):
    car_number = 0
    plt.figure(figsize=(12, 4))
    for image_slot in range(12):
        ax = plt.subplot(2, 6, image_slot + 1)
        if image_slot % 2 == 0:
            print(car_number)
            plt.imshow((images[car_number])) 
            class_name = 'Image'
        else:
            plt.imshow(masks[car_number], cmap = 'gray')
            #plt.colorbar()
            class_name = 'Mask'
            car_number += 1            
        plt.title(class_name)
        plt.axis("off")
    plt.show()